In [ ]:
#| default_exp client

# basic client functionality

> basic client operations not specific to the `rebroadcastr` client

This notebook will implement and test a barebones client that can easily manage connections, add subscriptions, get events, and publish events.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import warnings
import json
import time
import os
import pprint
import sqlite3
import appdirs
import pandas as pd
from pathlib import Path
from nostr.message_type import ClientMessageType
from nostr.message_pool import EventMessage,\
    NoticeMessage, EndOfStoredEventsMessage
from nostr.filter import Filter, Filters
from nostr.event import Event, EventKind
from rebroadcastr.nostr import PrivateKey, PublicKey,\
    RelayManager, MessagePool

from fastcore.utils import patch

In [ ]:
#| export

class Client:
    def __init__(self, public_key_hex: str = None, private_key_hex: str = None,
                 relay_urls: list = None, ssl_options: dict = {},
                 first_response_only: bool = True):
        """A basic framework for common operations that a nostr client will
        need to execute.

        Args:
            public_key_hex (str, optional): public key to initiate client
            private_key_hex (str, optional): private key to log in with public key.
                Defaults to None, in which case client is effectively read only
            relay_urls (list, optional): provide a list of relay urls.
                Defaults to None, in which case a default list will be used.
            ssl_options (dict, optional): ssl options for websocket connection
                Defaults to empty dict
            allow_duplicates (bool, optional): whether or not to allow duplicate
                event ids into the queue from multiple relays. This isn't fully
                working yet. Defaults to False.
        """
        self._is_connected = False
        self.ssl_options = ssl_options
        self.first_response_only = first_response_only
        self.set_account(public_key_hex=public_key_hex,
                          private_key_hex=private_key_hex)
        if relay_urls is None:
            relay_urls = [
                'wss://nostr-2.zebedee.cloud',
                'wss://relay.damus.io',
                'wss://brb.io',
                'wss://nostr-2.zebedee.cloud',
                'wss://rsslay.fiatjaf.com',
                'wss://nostr-relay.wlvs.space',
                'wss://nostr.orangepill.dev',
                'wss://nostr.oxtr.dev'
            ]
        else:
            pass
        self.relay_manager = RelayManager(first_response_only=self.first_response_only)
        self.events_table_name = 'events'
        self.init_db()
        self.set_relays(relay_urls=relay_urls)
        self.load_existing_event_ids()

    def set_account(self, public_key_hex: str = None, private_key_hex: str = None) -> None:
        """logic to set public and private keys

        Args:
            public_key_hex (str, optional): if only public key is provided, operations
                that require a signature will fail. Defaults to None.
            private_key_hex (str, optional): _description_. Defaults to None.

        Raises:
            ValueException: if the private key and public key are both provided but
                don't match
        """
        self.private_key = None
        self.public_key = None
        if private_key_hex is None:
            self.private_key = self._request_private_key_hex()
        else:
            self.private_key = PrivateKey.from_hex(private_key_hex)

        if public_key_hex is None:
            self.public_key = self.private_key.public_key
        else:
            self.public_key = PublicKey.from_hex(public_key_hex)
        public_key_hex = self.public_key.hex()
        
        if public_key_hex != self.private_key.public_key.hex():
            self.public_key = PublicKey.from_hex(public_key_hex)
            self.private_key = None
        print(f'logged in as public key\n'
              f'\tbech32: {self.public_key.bech32()}\n'
              f'\thex: {self.public_key.hex()}')
    
    def _request_private_key_hex(self) -> str:
        """method to request private key. this method should be overwritten
        when building out a UI

        Returns:
            PrivateKey: the new private_key object for the client. will also
                be set in place at self.private_key
        """
        self.private_key = PrivateKey()
        return self.private_key
    
    @property
    def db_conn(self):
        data_dir = Path(appdirs.user_data_dir('python-nostr'))
        data_dir.mkdir(exist_ok=True)
        return sqlite3.Connection(data_dir / f'{self.public_key.bech32()}.sqlite')
    
    def init_db(self):
        with self.db_conn as con:
            con.execute(f'CREATE TABLE IF NOT EXISTS {self.events_table_name} '
                        '(id char, pubkey char, created_at int, kind int, '
                        'tags char, content char, sig char,'
                        'subscription_id char, url char);')
            con.execute(f'CREATE INDEX IF NOT EXISTS ID_IDX ON {self.events_table_name}(ID);')
            con.execute(f'CREATE INDEX IF NOT EXISTS URL_IDX ON {self.events_table_name}(URL);')
        
    def set_relays(self, relay_urls: list = None):
        relays_to_add = set(relay_urls) - set(self.relay_manager.relays.keys())
        relays_to_remove = set(self.relay_manager.relays.keys()) - set(relay_urls)
        for url in relays_to_remove:
            self.relay_manager.remove_relay(url)
        was_connected = self.relay_manager._is_connected
        for url in relays_to_add:
            self.relay_manager.add_relay(url=url)
            if was_connected:
                self.relay_manager[url].open_connections()
        if was_connected:
            self.relay_manager.remove_closed_relays()

    def load_existing_event_ids(self):
        ids = pd.read_sql(sql='select id, url from events',
                          con=self.db_conn)
        if self.first_response_only:
            ids = ids['id']
        else:
            ids = ids['id'] + ':' + ids['url']
        self.relay_manager.message_pool._unique_objects = set(ids.to_list())

### tests
test setting account by private key or public key and handles as expected - note that some uses like the `rebroadcastr` client don't need a private key since we aren't creating new events

In [ ]:
import ssl

In [ ]:
private_key = PrivateKey()
public_key = private_key.public_key

# load client with no keys
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE})

# load client with public key only
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, public_key_hex=public_key.hex())
assert client.public_key.hex() == public_key.hex()
assert client.private_key is None

# load client with public key and private key
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, public_key_hex=public_key.hex(), private_key_hex=private_key.hex())
assert client.public_key.hex() == public_key.hex()
assert client.private_key.hex() == private_key.hex()

# load client with private key only
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, private_key_hex=private_key.hex())
assert client.public_key.hex() == public_key.hex()
assert client.private_key.hex() == private_key.hex()


logged in as public key
	bech32: npub1t5hx0py7u60zrhtq8wt6ykry3666vanjg9v92n40ddd08dmkc08qs22q2d
	hex: 5d2e67849ee69e21dd603b97a258648eb5a676724158554eaf6b5af3b776c3ce
logged in as public key
	bech32: npub1w8j8y4q69ycmgf38kclcvwvphanm0e830y6pjtd27j2j9nsp8deqak4l7z
	hex: 71e472541a2931b42627b63f863981bf67b7e4f17934192daaf49522ce013b72
logged in as public key
	bech32: npub1w8j8y4q69ycmgf38kclcvwvphanm0e830y6pjtd27j2j9nsp8deqak4l7z
	hex: 71e472541a2931b42627b63f863981bf67b7e4f17934192daaf49522ce013b72
logged in as public key
	bech32: npub1w8j8y4q69ycmgf38kclcvwvphanm0e830y6pjtd27j2j9nsp8deqak4l7z
	hex: 71e472541a2931b42627b63f863981bf67b7e4f17934192daaf49522ce013b72


test adding and removing relays while disconnected

In [ ]:
relay_urls_1 = [
                'wss://nostr-2.zebedee.cloud',
                'wss://rsslay.fiatjaf.com',
                'wss://nostr-relay.wlvs.space',
                'wss://nostr.orangepill.dev',
                'wss://nostr.oxtr.dev'
            ]
relay_urls_2 = [
                'wss://relay.damus.io',
                'wss://brb.io',
                'wss://nostr-2.zebedee.cloud',
                'wss://rsslay.fiatjaf.com',
            ]
client = Client(private_key_hex=private_key.hex(), relay_urls=relay_urls_1)
assert set(relay_urls_1) == set(client.relay_manager.relays.keys())
client.set_relays(relay_urls_2)
assert set(relay_urls_2) == set(client.relay_manager.relays.keys())


logged in as public key
	bech32: npub1w8j8y4q69ycmgf38kclcvwvphanm0e830y6pjtd27j2j9nsp8deqak4l7z
	hex: 71e472541a2931b42627b63f863981bf67b7e4f17934192daaf49522ce013b72


## adding connection methods

In [ ]:
#| export

@patch
def __enter__(self: Client):
    """context manager to allow processing a connected client
    within a `with` statement

    Returns:
        self: a `with` statement returns this object as it's assignment
        so that the client can be instantiated and used within
        the `with` statement.
    """
    self.connect()
    return self

@patch
def __exit__(self: Client, type, value, traceback):
    """closes the connections when exiting the `with` context

    arguments are currently unused, but could be use to control
    client behavior on error.

    Args:
        type (_type_): _description_
        value (_type_): _description_
        traceback (_type_): _description_
    """
    self.disconnect()
    return traceback

@patch
def connect(self: Client) -> None:
    self.relay_manager.open_connections(self.ssl_options)

@patch
def disconnect(self: Client) -> None:
    self.relay_manager.close_connections()


NameError: name 'patch' is not defined

In [ ]:
# relay_urls_1 = [
#                 'wss://nostr-2.zebedee.cloud',
#                 'wss://rsslay.fiatjaf.com',
#                 'wss://nostr-relay.wlvs.space',
#                 'wss://nostr.orangepill.dev',
#                 'wss://nostr.oxtr.dev'
#             ]
# relay_urls_2 = [
#                 'wss://relay.damus.io',
#                 'wss://brb.io',
#                 'wss://nostr-2.zebedee.cloud',
#                 'wss://rsslay.fiatjaf.com',
#             ]
# client = Client(private_key_hex=private_key.hex(), relay_urls=relay_urls_1)
# with client.relay_manager.connection():
# assert set(relay_urls_1) == set(client.relay_manager.relays.keys())
# client.set_relays(relay_urls_2)
# assert set(relay_urls_2) == set(client.relay_manager.relays.keys())


In [ ]:

    
#     def __enter__(self):
#         """context manager to allow processing a connected client
#         within a `with` statement

#         Returns:
#             self: a `with` statement returns this object as it's assignment
#             so that the client can be instantiated and used within
#             the `with` statement.
#         """
#         self.connect()
#         self.run()
#         return self

#     def __exit__(self, type, value, traceback):
#         """closes the connections when exiting the `with` context

#         arguments are currently unused, but could be use to control
#         client behavior on error.

#         Args:
#             type (_type_): _description_
#             value (_type_): _description_
#             traceback (_type_): _description_
#         """
#         self.disconnect()
#         return traceback

#     def connect(self) -> None:
#         self.relay_manager.open_connections(self.ssl_options)
#         time.sleep(2)
#         for url, connected in self.connection_statuses.items():
#             if not connected:
#                 warnings.warn(
#                     f'could not connect to {url}... removing relay.'
#                 )
#                 self.relay_manager.remove_relay(url=url)
#         assert all(self.connection_statuses.values())
#         self._is_connected = True
    
#     def disconnect(self) -> None:
#         time.sleep(2)
#         self.relay_manager.close_connections()
#         self._is_connected = False

#     def _event_handler(self, event_msg: EventMessage) -> pd.DataFrame:
#         """a hidden method used to handle event outputs
#         from a relay. This can be overwritten to store events
#         to a db for example.

#         Args:
#             event_msg (EventMessage): Event message returned from relay
#         """
#         event = event_msg.event
#         rebroadcast_id = f'{event.id}:{event_msg.url}'
#         already_received = rebroadcast_id in \
#                            self.relay_manager.message_pool._unique_objects
#         for relay in self.relay_manager:
#             if event_msg.url != relay.url and not already_received:
#                 self.to_rebroadcast[relay.url].update({event.id: event})
#         if event.id in self.to_rebroadcast[event_msg.url].keys():
#             del self.to_rebroadcast[event_msg.url][event.id]
#         data = event.to_json_object()
#         data['tags'] = str(data['tags'])
#         data.update({'subscription_id': event_msg.subscription_id,
#                      'url': event_msg.url})
#         return pd.DataFrame.from_records(
#                 [data]
#             )

#     @staticmethod
#     def _notice_handler(notice_msg: NoticeMessage):
#         """a hidden method used to handle notice outputs
#         from a relay. This can be overwritten to display notices
#         differently - should be warnings or errors?

#         Args:
#             notice_msg (NoticeMessage): Notice message returned from relay
#         """
#         warnings.warn(f'{notice_msg.url}:\n\t{notice_msg.content}')

#     @staticmethod
#     def _eose_handler(eose_msg: EndOfStoredEventsMessage):
#         """a hidden method used to handle notice outputs
#         from a relay. This can be overwritten to display notices
#         differently - should be warnings or errors?

#         Args:
#             notice_msg (EndOfStoredEventsMessage): Message from relay
#                 to signify the last event in a subscription has been
#                 provided.
#         """
#         print(f'end of subscription: {eose_msg.subscription_id} received.')

#     def get_events_from_relay(self):
#         """calls the _event_handler method on all events from relays
#         """
#         self.events = []
#         while self.relay_manager.message_pool.has_events():
#             event_msg = self.relay_manager.message_pool.get_event()
#             self.events.append(
#                 self._event_handler(event_msg=event_msg)
#             )
#         if len(self.events) > 0:
#             event_df = pd.concat(self.events)
#             print(f'writing {len(event_df)} event(s)')
#             with self.db_conn as con:
#                 event_df.to_sql(name=self.events_table_name, con=con,
#                                 if_exists='append', index=False,)
            
#     def get_notices_from_relay(self):
#         """calls the _notice_handler method on all notices from relays
#         """
#         while self.relay_manager.message_pool.has_notices():
#             notice_msg = self.relay_manager.message_pool.get_notice()
#             self._notice_handler(notice_msg=notice_msg)

#     def get_eose_from_relay(self):
#         """calls the _eose_handler end of subsribtion events from relays
#         """
#         while self.relay_manager.message_pool.has_eose_notices():
#             eose_msg = self.relay_manager.message_pool.get_eose_notice()
#             self._eose_handler(eose_msg=eose_msg)

#     def publish_request(self, subscription_id: str, request_filters: Filters) -> None:
#         """publishes a request from a subscription id and a set of filters. Filters
#         can be defined using the request_by_custom_filter method or from a list of
#         preset filters (as of yet to be created):

#         Args:
#             subscription_id (str): subscription id to be sent to relau
#             request_filters (Filters): list of filters for a subscription
#         """
#         request = [ClientMessageType.REQUEST, subscription_id]
#         request.extend(request_filters.to_json_array())
#         message = json.dumps(request)
#         self.relay_manager.add_subscription(
#             subscription_id, request_filters
#             )
#         self.relay_manager.publish_message(message)
#         time.sleep(1)
#         self.get_notices_from_relay()
    
#     def publish_event(self, event: Event) -> None:
#         """publish an event and immediately checks for a notice
#         from the relay in case of an invalid event

#         Args:
#             event (Event): _description_
#         """
#         event.sign(self.private_key.hex())
#         message = json.dumps([ClientMessageType.EVENT, event.to_json_object()])
#         self.relay_manager.publish_message(message)
#         time.sleep(1)
#         self.get_notices_from_relay()

#     def request_by_custom_filter(self, subscription_id, **filter_kwargs) -> None:
#         """make a relay request from kwargs for a single Filter object
#         as defined in python-nostr.filter.Filter

#         Args:
#             subscription_id (_type_): _description_
#         Kwargs to follow python-nostr.filter.Filter
#         """
#         custom_request_filters = Filters([Filter(**filter_kwargs)])
#         self.publish_request(
#             subscription_id=subscription_id,
#             request_filters=custom_request_filters
#         )

# ######################### publish methods #######################
# # establishing methods for all possible events outlined here:   #
# # https://github.com/nostr-protocol/nips#event-kinds            #
# #################################################################

#     def publish_metadata(self) -> None:
#         raise NotImplementedError()
    
#     def publish_text_note(self, text: str) -> None:
#         """publish a text note to relays

#         Args:
#             text (str): text for nostr note to be published
#         """
#         # TODO: need regex parsing to handle @
#         event = Event(public_key=self.public_key.hex(),
#                       content=text,
#                       kind=EventKind.TEXT_NOTE)
#         self.publish_event(event=event)

#     def publish_recommended_relay(self) -> None:
#         raise NotImplementedError()

#     def publish_deletion(self, event_id: str, reason: str) -> None:
#         """delete a single event by id

#         Args:
#             event_id (str): event id/hash
#             reason (str): a reason for deletion provided by the user
#         """
#         event = Event(public_key=self.public_key.hex(),
#                       kind=EventKind.DELETE,
#                       content=reason,
#                       tags=[['e', event_id]])
#         self.publish_event(event=event)

#     def publish_reaction(self) -> None:
#         raise NotImplementedError()

#     def publish_channel(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_metadata(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_message(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_hide_message(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_mute_user(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_metadata(self) -> None:
#         raise NotImplementedError()

#     def send_encrypted_message(self) -> None:
#         warnings.warn('''the current implementation of messages should be used with caution
#                       see https://github.com/nostr-protocol/nips/issues/107''')
#         raise NotImplementedError()


# ################### filter building methods #####################
# #  this section is reserved for methods used to build filters   #
# #  that can be used in requests to relays. things like get      #
# #  all posts from a list of user ids with a limit of x.         #
# #                                                               #
# #################################################################
# # TODO: BUILD methods and a static filter dict

#     def run(self):
#         self.init_subscriptions()
#         while True != False:

#             self.start_time = time.time() - self.interval
#             self.to_rebroadcast = \
#                 {relay.url: {} for relay in self.relay_manager}
#             self.get_events_from_relay()
#             # for relay in self.relay_manager:
#             #     print(relay.url)
#             #     print(f'\t{len(self.to_rebroadcast[relay.url])} events to rebroadcast',
#             #         self.to_rebroadcast[relay.url])
#             self.rebroadcast_events()
#             print('waiting for next update')
#             time.sleep(self.interval)
#             # self.

#     def rebroadcast_events(self):
#         for url, to_rebroadcast in self.to_rebroadcast.items():

#             print(f'rebroadcasting {len(to_rebroadcast)} events to {url}')
#             relay = self.relay_manager.relays[url]
#             for event in to_rebroadcast.values():
#                 if event.created_at > self.start_time:
#                     print(event.content)
#                     message = json.dumps([ClientMessageType.EVENT, event.to_json_object()])
#                     relay.publish(message)
#             time.sleep(1)
#         self.get_notices_from_relay()

# # class TextInputClient(Client):
# #     '''
# #     a simple client that can be run as
# #     ```
# #     with TextInputClient():
# #         pass
# #     '''
# #     ## changing a few key methods that are used in the base class ##

# #     def __init__(self, *args, **kwargs):
# #         """adding a message store where we
# #         can store messages using the _event_handler method
# #         """
# #         super().__init__(*args, **kwargs)
# #         self.message_store = {}

# #     def __enter__(self):
# #         super().__enter__()
# #         self.run()
# #         return self
    
# #     def _request_private_key_hex(self) -> PrivateKey:
# #         """the only requirement of this method is that it
# #         needs to in some way set the self.private_key
# #         attribute to an instance of PrivateKey
# #         """
# #         user_hex = input('please enter a private key hex')
# #         if user_hex.strip() == '':
# #             user_hex = 'x'
# #         try:
# #             self.private_key = PrivateKey.from_hex(user_hex)
# #             print('successfully loaded')
# #         except:
# #             print('could not generate private key from input. '
# #                   'generating a new random key')
# #             self.private_key = PrivateKey()
# #             print(f'generated new private key: {self.private_key.hex()}')
# #         return self.private_key

# #     def _event_handler(self, event_msg) -> None:
# #         event = event_msg.event
# #         print(f'author: {event.public_key}\n'
# #               f'event id: {event.id}\n'
# #               f'url: {event_msg.url}\n'
# #               f'\t{event.content}')
# #         self.message_store.update({f'{event_msg.url}:{event.id}': event_msg})

# #     ########## adding a couple methods used to run the app! ###########
# #     ## these could be replaced by a more complex interface in theory ##

# #     def run(self) -> None:
# #         cmd = 'start'
# #         while cmd not in ['exit', 'x', '0']:
# #             if cmd != 'start':
# #                 self.execute(cmd)
# #             menu = '''select a command:
# #                 \t0\tE(x)it
# #                 \t1\tpublish note
# #                 \t2\tget last 10 notes by you
# #                 \t3\tget last 10 from hex of author
# #                 \t4\tdelete an event
# #                 \t5\tcheck deletions
# #                 \t6\tget metadata by hex of user
# #                 \t7\tcheck event
# #                 \t8\tget recommended server
# #                 \t9\tget contacts
# #                 \t10\tprint relays
# #                 \t11\tadd relay
# #                 '''
# #             print(menu)
# #             cmd = input('see output for choices').lower()
# #         print('exiting')
    
# #     def execute(self, cmd) -> None:
# #         if cmd == '1':
# #             text_note = input(f'Enter a text note:\n')
# #             print(f'note:\n\n{text_note}')
# #             response = input('is the note output below correct? y/n').lower()
# #             if response in ['y', 'yes']:
# #                 self.publish_text_note(text_note)
# #                 print('published.')
# #             else:
# #                 print('returning to menu')
# #         elif cmd == '2':
# #             author = self.public_key.hex()
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10',
# #                 kinds=[EventKind.TEXT_NOTE],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '3':
# #             author = input('who?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10',
# #                 kinds=[EventKind.TEXT_NOTE],
# #                 authors=[self.public_key.hex()],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '4':
# #             event_id = input('which event id?')
# #             reason = input('please give a reason')
# #             self.publish_deletion(event_id=event_id, reason=reason)
        
# #         elif cmd == '5':
# #             author = self.public_key.hex()
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10deletes',
# #                 kinds=[EventKind.DELETE],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)

# #         elif cmd == '6':
# #             author = input('who?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10metadata',
# #                 kinds=[EventKind.SET_METADATA],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)

# #         elif cmd == '7':
# #             event_id = input('event id to check?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{event_id}_single_event',
# #                 kinds=[EventKind.TEXT_NOTE],
# #                 ids=[event_id],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '8':
# #             author = input('user to check?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_recommended_server',
# #                 kinds=[EventKind.RECOMMEND_RELAY],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '9':
# #             author = input('user to check?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_contacts',
# #                 kinds=[EventKind.CONTACTS],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '10':
# #             print(self.relay_urls)
# #         elif cmd == '11':
# #             relay_url = input('what is the relay url')
# #             self.set_relays(self.relay_urls + [relay_url])
# #             print(f'connection status: {self.connection_statuses}')

# #         else:
# #             print('command not found. returning to menu')

In [ ]:
# raise

In [ ]:


# class Rebroadcaster:

#     def __init__(self, public_key_hex: str = None, relay_urls: list = None,
#                  ssl_options: dict = {}, interval: int = 60):
#         """A basic framework for common operations that a nostr client will
#         need to execute.

#         Args:
#             public_key_hex (str, optional): public key to initiate client
#             private_key_hex (str, optional): private key to log in with public key.
#                 Defaults to None, in which case client is effectively read only
#             relay_urls (list, optional): provide a list of relay urls.
#                 Defaults to None, in which case a default list will be used.
#             ssl_options (dict, optional): ssl options for websocket connection
#                 Defaults to empty dict
#             allow_duplicates (bool, optional): whether or not to allow duplicate
#                 event ids into the queue from multiple relays. This isn't fully
#                 working yet. Defaults to False.
#         """
#         self.interval = interval
#         self._is_connected = False
#         self.ssl_options = ssl_options
#         self.first_response_only = False # we need to see the event across all relays
#         self.events_table_name = 'events'

#         self.set_account(public_key_hex=public_key_hex)
#         self.init_db()
#         if relay_urls is None:
#             relay_urls = [
#                 'wss://nostr-2.zebedee.cloud',
#                 'wss://relay.damus.io',
#                 'wss://brb.io',
#                 'wss://nostr-2.zebedee.cloud',
#                 'wss://rsslay.fiatjaf.com',
#                 'wss://nostr-relay.wlvs.space',
#                 'wss://nostr.orangepill.dev',
#                 'wss://nostr.oxtr.dev'
#             ]
#         else:
#             pass
#         self.set_relays(relay_urls=relay_urls)
    
#     def __enter__(self):
#         """context manager to allow processing a connected client
#         within a `with` statement

#         Returns:
#             self: a `with` statement returns this object as it's assignment
#             so that the client can be instantiated and used within
#             the `with` statement.
#         """
#         self.connect()
#         self.run()
#         return self

#     def __exit__(self, type, value, traceback):
#         """closes the connections when exiting the `with` context

#         arguments are currently unused, but could be use to control
#         client behavior on error.

#         Args:
#             type (_type_): _description_
#             value (_type_): _description_
#             traceback (_type_): _description_
#         """
#         self.disconnect()
#         return traceback

#     def connect(self) -> None:
#         self.relay_manager.open_connections(self.ssl_options)
#         time.sleep(2)
#         for url, connected in self.connection_statuses.items():
#             if not connected:
#                 warnings.warn(
#                     f'could not connect to {url}... removing relay.'
#                 )
#                 self.relay_manager.remove_relay(url=url)
#         assert all(self.connection_statuses.values())
#         self._is_connected = True
    
#     def disconnect(self) -> None:
#         time.sleep(2)
#         self.relay_manager.close_connections()
#         self._is_connected = False
    
#     def init_subscriptions(self) -> None:
#         author = self.public_key.hex()
#         self.request_by_custom_filter(
#                 subscription_id=f'{author}_contacts',
#                 kinds=[EventKind.CONTACTS],
#                 authors=[author],
#                 )
#         self.request_by_custom_filter(
#             subscription_id=f'{author}_last10metadata',
#             kinds=[EventKind.SET_METADATA],
#             authors=[author],
#             )
#         self.request_by_custom_filter(
#             subscription_id=f'{author}_last10',
#             kinds=[EventKind.TEXT_NOTE],
#             authors=[author],
#         )

    
#     @property
#     def relay_urls(self) -> list:
#         return [relay.url for relay in self.relay_manager]
    
#     @property
#     def connection_statuses(self) -> dict:
#         """gets the url and connection statuses of relays

#         Returns:
#             dict: bool of connection statuses
#         """
#         statuses = [relay.is_connected for relay in self.relay_manager]
#         return dict(zip(self.relay_urls, statuses))

#     @property
#     def db_conn(self):
#         return sqlite3.Connection(f'./{self.public_key.bech32()}.sqlite')
    
#     def init_db(self):
#         with self.db_conn as con:
#             con.execute(f'CREATE TABLE IF NOT EXISTS {self.events_table_name} '
#                         '(id char, pubkey char, created_at int, kind int, '
#                         'tags char, content char, sig char,'
#                         'subscription_id char, url char);')
#             con.execute(f'CREATE INDEX IF NOT EXISTS ID_IDX ON {self.events_table_name}(ID);')
#             con.execute(f'CREATE INDEX IF NOT EXISTS URL_IDX ON {self.events_table_name}(URL);')


#     def set_account(self, public_key_hex: str = None, private_key_hex: str = None) -> None:
#         """logic to set public and private keys

#         Args:
#             public_key_hex (str, optional): if only public key is provided, operations
#                 that require a signature will fail. Defaults to None.
#             private_key_hex (str, optional): _description_. Defaults to None.

#         Raises:
#             ValueException: if the private key and public key are both provided but
#                 don't match
#         """
#         self.private_key = None
#         self.public_key = None
#         if private_key_hex is None:
#             self.private_key = self._request_private_key_hex()
#         else:
#             self.private_key = PrivateKey.from_hex(private_key_hex)

#         if public_key_hex is None:
#             self.public_key = self.private_key.public_key
#         else:
#             self.public_key = PublicKey.from_hex(public_key_hex)
#         public_key_hex = self.public_key.hex()
        
#         if public_key_hex != self.private_key.public_key.hex():
#             self.public_key = PublicKey.from_hex(public_key_hex)
#             self.private_key = None
#         print(f'logged in as public key\n'
#               f'\tbech32: {self.public_key.bech32()}\n'
#               f'\thex: {self.public_key.hex()}')
    
#     def _request_private_key_hex(self) -> str:
#         """method to request private key. this method should be overwritten
#         when building out a UI

#         Returns:
#             PrivateKey: the new private_key object for the client. will also
#                 be set in place at self.private_key
#         """
#         self.private_key = PrivateKey()
#         return self.private_key
        
#     def set_relays(self, relay_urls: list = None):
#         was_connected = self._is_connected
#         if self._is_connected:
#             self.disconnect()
#         self.relay_manager = RelayManager(first_response_only=self.first_response_only)
#         ids = pd.read_sql(sql='select id, url from events',
#                           con=self.db_conn)
#         ids = ids.id.astype(str) + ':' + ids.url.astype(str)
#         self.relay_manager.message_pool._unique_objects = set(ids.to_list())
#         for url in relay_urls:
#             self.relay_manager.add_relay(url=url)
#         if was_connected:
#             self.connect()

#     def _event_handler(self, event_msg: EventMessage) -> pd.DataFrame:
#         """a hidden method used to handle event outputs
#         from a relay. This can be overwritten to store events
#         to a db for example.

#         Args:
#             event_msg (EventMessage): Event message returned from relay
#         """
#         event = event_msg.event
#         rebroadcast_id = f'{event.id}:{event_msg.url}'
#         already_received = rebroadcast_id in \
#                            self.relay_manager.message_pool._unique_objects
#         for relay in self.relay_manager:
#             if event_msg.url != relay.url and not already_received:
#                 self.to_rebroadcast[relay.url].update({event.id: event})
#         if event.id in self.to_rebroadcast[event_msg.url].keys():
#             del self.to_rebroadcast[event_msg.url][event.id]
#         data = event.to_json_object()
#         data['tags'] = str(data['tags'])
#         data.update({'subscription_id': event_msg.subscription_id,
#                      'url': event_msg.url})
#         return pd.DataFrame.from_records(
#                 [data]
#             )

#     @staticmethod
#     def _notice_handler(notice_msg: NoticeMessage):
#         """a hidden method used to handle notice outputs
#         from a relay. This can be overwritten to display notices
#         differently - should be warnings or errors?

#         Args:
#             notice_msg (NoticeMessage): Notice message returned from relay
#         """
#         warnings.warn(f'{notice_msg.url}:\n\t{notice_msg.content}')

#     @staticmethod
#     def _eose_handler(eose_msg: EndOfStoredEventsMessage):
#         """a hidden method used to handle notice outputs
#         from a relay. This can be overwritten to display notices
#         differently - should be warnings or errors?

#         Args:
#             notice_msg (EndOfStoredEventsMessage): Message from relay
#                 to signify the last event in a subscription has been
#                 provided.
#         """
#         print(f'end of subscription: {eose_msg.subscription_id} received.')

#     def get_events_from_relay(self):
#         """calls the _event_handler method on all events from relays
#         """
#         self.events = []
#         while self.relay_manager.message_pool.has_events():
#             event_msg = self.relay_manager.message_pool.get_event()
#             self.events.append(
#                 self._event_handler(event_msg=event_msg)
#             )
#         if len(self.events) > 0:
#             event_df = pd.concat(self.events)
#             print(f'writing {len(event_df)} event(s)')
#             with self.db_conn as con:
#                 event_df.to_sql(name=self.events_table_name, con=con,
#                                 if_exists='append', index=False,)
            
#     def get_notices_from_relay(self):
#         """calls the _notice_handler method on all notices from relays
#         """
#         while self.relay_manager.message_pool.has_notices():
#             notice_msg = self.relay_manager.message_pool.get_notice()
#             self._notice_handler(notice_msg=notice_msg)

#     def get_eose_from_relay(self):
#         """calls the _eose_handler end of subsribtion events from relays
#         """
#         while self.relay_manager.message_pool.has_eose_notices():
#             eose_msg = self.relay_manager.message_pool.get_eose_notice()
#             self._eose_handler(eose_msg=eose_msg)

#     def publish_request(self, subscription_id: str, request_filters: Filters) -> None:
#         """publishes a request from a subscription id and a set of filters. Filters
#         can be defined using the request_by_custom_filter method or from a list of
#         preset filters (as of yet to be created):

#         Args:
#             subscription_id (str): subscription id to be sent to relau
#             request_filters (Filters): list of filters for a subscription
#         """
#         request = [ClientMessageType.REQUEST, subscription_id]
#         request.extend(request_filters.to_json_array())
#         message = json.dumps(request)
#         self.relay_manager.add_subscription(
#             subscription_id, request_filters
#             )
#         self.relay_manager.publish_message(message)
#         time.sleep(1)
#         self.get_notices_from_relay()
    
#     def publish_event(self, event: Event) -> None:
#         """publish an event and immediately checks for a notice
#         from the relay in case of an invalid event

#         Args:
#             event (Event): _description_
#         """
#         event.sign(self.private_key.hex())
#         message = json.dumps([ClientMessageType.EVENT, event.to_json_object()])
#         self.relay_manager.publish_message(message)
#         time.sleep(1)
#         self.get_notices_from_relay()

#     def request_by_custom_filter(self, subscription_id, **filter_kwargs) -> None:
#         """make a relay request from kwargs for a single Filter object
#         as defined in python-nostr.filter.Filter

#         Args:
#             subscription_id (_type_): _description_
#         Kwargs to follow python-nostr.filter.Filter
#         """
#         custom_request_filters = Filters([Filter(**filter_kwargs)])
#         self.publish_request(
#             subscription_id=subscription_id,
#             request_filters=custom_request_filters
#         )

# ######################### publish methods #######################
# # establishing methods for all possible events outlined here:   #
# # https://github.com/nostr-protocol/nips#event-kinds            #
# #################################################################

#     def publish_metadata(self) -> None:
#         raise NotImplementedError()
    
#     def publish_text_note(self, text: str) -> None:
#         """publish a text note to relays

#         Args:
#             text (str): text for nostr note to be published
#         """
#         # TODO: need regex parsing to handle @
#         event = Event(public_key=self.public_key.hex(),
#                       content=text,
#                       kind=EventKind.TEXT_NOTE)
#         self.publish_event(event=event)

#     def publish_recommended_relay(self) -> None:
#         raise NotImplementedError()

#     def publish_deletion(self, event_id: str, reason: str) -> None:
#         """delete a single event by id

#         Args:
#             event_id (str): event id/hash
#             reason (str): a reason for deletion provided by the user
#         """
#         event = Event(public_key=self.public_key.hex(),
#                       kind=EventKind.DELETE,
#                       content=reason,
#                       tags=[['e', event_id]])
#         self.publish_event(event=event)

#     def publish_reaction(self) -> None:
#         raise NotImplementedError()

#     def publish_channel(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_metadata(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_message(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_hide_message(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_mute_user(self) -> None:
#         raise NotImplementedError()

#     def publish_channel_metadata(self) -> None:
#         raise NotImplementedError()

#     def send_encrypted_message(self) -> None:
#         warnings.warn('''the current implementation of messages should be used with caution
#                       see https://github.com/nostr-protocol/nips/issues/107''')
#         raise NotImplementedError()


# ################### filter building methods #####################
# #  this section is reserved for methods used to build filters   #
# #  that can be used in requests to relays. things like get      #
# #  all posts from a list of user ids with a limit of x.         #
# #                                                               #
# #################################################################
# # TODO: BUILD methods and a static filter dict

#     def run(self):
#         self.init_subscriptions()
#         while True != False:

#             self.start_time = time.time() - self.interval
#             self.to_rebroadcast = \
#                 {relay.url: {} for relay in self.relay_manager}
#             self.get_events_from_relay()
#             # for relay in self.relay_manager:
#             #     print(relay.url)
#             #     print(f'\t{len(self.to_rebroadcast[relay.url])} events to rebroadcast',
#             #         self.to_rebroadcast[relay.url])
#             self.rebroadcast_events()
#             print('waiting for next update')
#             time.sleep(self.interval)
#             # self.

#     def rebroadcast_events(self):
#         for url, to_rebroadcast in self.to_rebroadcast.items():

#             print(f'rebroadcasting {len(to_rebroadcast)} events to {url}')
#             relay = self.relay_manager.relays[url]
#             for event in to_rebroadcast.values():
#                 if event.created_at > self.start_time:
#                     print(event.content)
#                     message = json.dumps([ClientMessageType.EVENT, event.to_json_object()])
#                     relay.publish(message)
#             time.sleep(1)
#         self.get_notices_from_relay()

# # class TextInputClient(Client):
# #     '''
# #     a simple client that can be run as
# #     ```
# #     with TextInputClient():
# #         pass
# #     '''
# #     ## changing a few key methods that are used in the base class ##

# #     def __init__(self, *args, **kwargs):
# #         """adding a message store where we
# #         can store messages using the _event_handler method
# #         """
# #         super().__init__(*args, **kwargs)
# #         self.message_store = {}

# #     def __enter__(self):
# #         super().__enter__()
# #         self.run()
# #         return self
    
# #     def _request_private_key_hex(self) -> PrivateKey:
# #         """the only requirement of this method is that it
# #         needs to in some way set the self.private_key
# #         attribute to an instance of PrivateKey
# #         """
# #         user_hex = input('please enter a private key hex')
# #         if user_hex.strip() == '':
# #             user_hex = 'x'
# #         try:
# #             self.private_key = PrivateKey.from_hex(user_hex)
# #             print('successfully loaded')
# #         except:
# #             print('could not generate private key from input. '
# #                   'generating a new random key')
# #             self.private_key = PrivateKey()
# #             print(f'generated new private key: {self.private_key.hex()}')
# #         return self.private_key

# #     def _event_handler(self, event_msg) -> None:
# #         event = event_msg.event
# #         print(f'author: {event.public_key}\n'
# #               f'event id: {event.id}\n'
# #               f'url: {event_msg.url}\n'
# #               f'\t{event.content}')
# #         self.message_store.update({f'{event_msg.url}:{event.id}': event_msg})

# #     ########## adding a couple methods used to run the app! ###########
# #     ## these could be replaced by a more complex interface in theory ##

# #     def run(self) -> None:
# #         cmd = 'start'
# #         while cmd not in ['exit', 'x', '0']:
# #             if cmd != 'start':
# #                 self.execute(cmd)
# #             menu = '''select a command:
# #                 \t0\tE(x)it
# #                 \t1\tpublish note
# #                 \t2\tget last 10 notes by you
# #                 \t3\tget last 10 from hex of author
# #                 \t4\tdelete an event
# #                 \t5\tcheck deletions
# #                 \t6\tget metadata by hex of user
# #                 \t7\tcheck event
# #                 \t8\tget recommended server
# #                 \t9\tget contacts
# #                 \t10\tprint relays
# #                 \t11\tadd relay
# #                 '''
# #             print(menu)
# #             cmd = input('see output for choices').lower()
# #         print('exiting')
    
# #     def execute(self, cmd) -> None:
# #         if cmd == '1':
# #             text_note = input(f'Enter a text note:\n')
# #             print(f'note:\n\n{text_note}')
# #             response = input('is the note output below correct? y/n').lower()
# #             if response in ['y', 'yes']:
# #                 self.publish_text_note(text_note)
# #                 print('published.')
# #             else:
# #                 print('returning to menu')
# #         elif cmd == '2':
# #             author = self.public_key.hex()
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10',
# #                 kinds=[EventKind.TEXT_NOTE],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '3':
# #             author = input('who?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10',
# #                 kinds=[EventKind.TEXT_NOTE],
# #                 authors=[self.public_key.hex()],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '4':
# #             event_id = input('which event id?')
# #             reason = input('please give a reason')
# #             self.publish_deletion(event_id=event_id, reason=reason)
        
# #         elif cmd == '5':
# #             author = self.public_key.hex()
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10deletes',
# #                 kinds=[EventKind.DELETE],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)

# #         elif cmd == '6':
# #             author = input('who?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_last10metadata',
# #                 kinds=[EventKind.SET_METADATA],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)

# #         elif cmd == '7':
# #             event_id = input('event id to check?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{event_id}_single_event',
# #                 kinds=[EventKind.TEXT_NOTE],
# #                 ids=[event_id],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '8':
# #             author = input('user to check?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_recommended_server',
# #                 kinds=[EventKind.RECOMMEND_RELAY],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '9':
# #             author = input('user to check?')
# #             self.request_by_custom_filter(
# #                 subscription_id=f'{author}_contacts',
# #                 kinds=[EventKind.CONTACTS],
# #                 authors=[author],
# #                 limit=10
# #                 )
# #             print(self.message_store)
# #         elif cmd == '10':
# #             print(self.relay_urls)
# #         elif cmd == '11':
# #             relay_url = input('what is the relay url')
# #             self.set_relays(self.relay_urls + [relay_url])
# #             print(f'connection status: {self.connection_statuses}')

# #         else:
# #             print('command not found. returning to menu')


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()